In [ ]:
 python -m pip install -U "scipy==1.4.*" "pulp==2.1"
 pulptest

In [1]:
import numpy as np 
from scipy.optimize import linprog
obj = [-1, -2]  # -1 = coefficient of x 
                # -2 = coeff. of y

lhs_ineq = [[ 2,  1],  # Red constraint left side
            [-4,  5],  # Blue constraint left side
            [ 1, -2]]  # Yellow constraint left side

rhs_ineq = [20,  # Red constraint right side
            10,  # Blue constraint right side
            2]  # Yellow constraint right side

lhs_eq = [[-1, 5]]  # Green constraint left side
rhs_eq = [15]       # Green constraint right side

bnd = [(0, float("inf")),  # Bounds of x
       (0, float("inf"))]  # Bounds of y
opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
             A_eq=lhs_eq, b_eq=rhs_eq, bounds=bnd,
         method="revised simplex")
opt
   
    


#opt.fun

#opt.success

#opt.x


     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

In [ ]:
# Above ^^ we use the "revised simplex" method 
# Other methods:
#     1. method="interior-point" selects the interior-point method. This option is set by default.
#     2. method="revised simplex" selects the revised two-phase simplex method.
#     3. method="simplex" selects the legacy two-phase simplex method.



# ** linprog() returns a data structure with these attributes: **

#     .con is the equality constraints residuals.

#     .fun is the objective function value at the optimum (if found).

#     .message is the status of the solution.

#     .nit is the number of iterations needed to finish the calculation.

#     .slack is the values of the slack variables, or the differences between the values of the left and right sides of the constraints.

#     .status is an integer between 0 and 4 that shows the status of the solution, such as 0 for when the optimal solution has been found.

#     .success is a Boolean that shows whether the optimal solution has been found.

#     .x is a NumPy array holding the optimal values of the decision variables.


Above ^^ we use the "revised simplex" method 
Other methods:
    1. method="interior-point" selects the interior-point method. This option is set by default.
    2. method="revised simplex" selects the revised two-phase simplex method.
    3. method="simplex" selects the legacy two-phase simplex method.
    
    
linprog() returns a data structure with these attributes:

    1 - .con is the equality constraints residuals.

    2 - .fun is the objective function value at the optimum (if found).

    3 - .message is the status of the solution.

    4 - .nit is the number of iterations needed to finish the calculation.

    5 - .slack is the values of the slack variables, or the differences between the values of the left and right sides of the constraints.

    6 - .status is an integer between 0 and 4 that shows the status of the solution, such as 0 for when the optimal solution has been found.

    7 - .success is a Boolean that shows whether the optimal solution has been found.

    8 - .x is a NumPy array holding the optimal values of the decision variables.


In [ ]:
## Problem (1) - Production planning problem

import numpy as np 
from scipy.optimize import linprog
import matplotlib.pyplot as plt

obj = [-300, -200]  # -1 = coefficient of x 
                # -2 = coeff. of y

lhs_ineq = [[ 2,  1],  
            [1,  2]]
            

rhs_ineq = [8, 
            8] 


bnd = [(0, float("inf")),  # Bounds of x
       (0, float("inf"))]  # Bounds of y


opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method="revised simplex")
opt

 

In [ ]:
# Production planning problem - using puLP

from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

expression = 2 * x + 4 * y
type(expression)


constraint = 2 * x + 4 * y >= 8
type(constraint)


# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
obj_func = x + 2 * y
model += obj_func

# Add the objective function to the model - Another method 
# model += x + 2 * y

# Add the objective function to the model - Method 3
# model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")
    
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")
    
model.variables()

model.variables()[0] is x

model.variables()[1] is y

model.solver



In [3]:
# Using GLPK and PuLP
# decision variables are *continous* 

from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK

# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve(solver=GLPK(msg=False))


print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")
    
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")
    
model.solver

status: 1, Optimal
objective: 16.81817
x: 7.72727
y: 4.54545
red_constraint: -1.0000000000509601e-05
blue_constraint: 18.181830000000005
yellow_constraint: 3.3636299999999997
green_constraint: -2.000000000279556e-05


In [4]:
## same problem as above - 
  # with decision variable x as an integer value. 
    
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK

model = LpProblem(name="small-problem", sense=LpMaximize)

x = LpVariable(name="x", lowBound=0, cat="Integer")
y = LpVariable(name="y", lowBound=0)

model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")
    
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")
    
model.solver 
# give us the solver which was used. 

status: 1, Optimal
objective: 15.8
x: 7
y: 4.4
red_constraint: -1.5999999999999996
blue_constraint: 16.0
yellow_constraint: 3.8000000000000007
green_constraint: 0.0


In [2]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from pulp import GLPK

# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

# Initialize the decision variables
x = LpVariable(name="x", lowBound=0)
y = LpVariable(name="y", lowBound=0)

# Add the constraints to the model
model += (2 * x + y <= 20, "red_constraint")
model += (4 * x - 5 * y >= -10, "blue_constraint")
model += (-x + 2 * y >= -2, "yellow_constraint")
model += (-x + 5 * y == 15, "green_constraint")

# Add the objective function to the model
model += lpSum([x, 2 * y])

# Solve the problem
status = model.solve(solver=GLPK(msg=False))
# Solve the problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")




status: 1, Optimal
objective: 16.81817
x: 7.72727
y: 4.54545
red_constraint: -1.0000000000509601e-05
blue_constraint: 18.181830000000005
yellow_constraint: 3.3636299999999997
green_constraint: -2.000000000279556e-05
